读取clinvar数据

1. 过滤致病等级
2. 过滤突变类型
3. 过滤基因组版本
4. 过滤MT染色体
5. 过滤na的碱基
7. 根据review status过滤可信度低的突变
6. 给标签
8. 写入variants_proessed.csv

In [ ]:
import pandas as pd
import os
pd.options.display.max_columns = None


clinvar_data_file = '/data/personal/zhenxy/clinvar/variant_summary.txt'
if not os.path.exists(clinvar_data_file):
    raise FileExistsError('The Clinvar Data File is not existed')
raw_df = pd.read_csv(clinvar_data_file, delimiter='\t')
raw_df.head(3)
raw_df = raw_df[(raw_df['ClinicalSignificance'] == 'Pathogenic') | (raw_df['ClinicalSignificance'] == 'Benign')]
raw_df = raw_df[raw_df['Type']=='single nucleotide variant']
raw_df = raw_df[raw_df['Assembly'] == 'GRCh38']
raw_df = raw_df[raw_df['Chromosome']!='MT']
raw_df = raw_df[raw_df['ReferenceAlleleVCF'] != 'na']
review_status = [
    'criteria provided, multiple submitters, no conflicts',
    'reviewed by expert panel',
    'practice guideline'
    ]
raw_df = raw_df[raw_df['ReviewStatus'].isin(review_status)]
raw_df['label'] = -1
raw_df['label'][raw_df['ClinicalSignificance'] == 'Pathogenic'] = 1
raw_df['label'][raw_df['ClinicalSignificance'] != 'Pathogenic'] = 0
processed_data_dir = '../data/clinvar/'
if  not os.path.exists(processed_data_dir):
    os.makedirs(processed_data_dir)
file_name = 'processed_variant_summary.txt'

raw_df.to_csv(os.path.join(processed_data_dir, file_name), sep='\t', index=False)

In [3]:
import pandas as pd
from pandarallel import pandarallel

pandarallel.initialize()

chrs = [str(i) for i in range(1, 23)]
chrs.extend(['X', 'Y'])
chr_seqs = {}
for chr in chrs:
    with open(r'/data/personal/zhenxy/ucsc/hg38/chr{}.txt'.format(chr)) as f:
        seq = f.readline()
        chr_seqs[chr] = seq 

print("genome sequences are loaded.")

def get_seq(row, seq_len):
    chr = row['Chromosome']
    seq = chr_seqs[chr]
    point_index = int(row['PositionVCF'])
    ref = row['ReferenceAlleleVCF']
    alt = row['AlternateAlleleVCF']
    left = seq[point_index-1-seq_len:point_index-1]
    right = seq[point_index: point_index+seq_len]
    if len(left)<seq_len:
        left = 'N' *(seq_len-len(left)) + left
    if len(right)<seq_len:
        right = right + 'N'*(seq_len-len(right))
    t1 = ''.join([left, ref, right])
    t2 = ''.join([left, alt, right])
    return t1, t2

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
genome sequences are loaded.


In [2]:
import pandas as pd
pd.options.display.max_columns = None

variant_df = pd.read_csv('../data/clinvar/processed_variant_summary.txt', sep='\t')
variant_df.head(5)


,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),nsv/esv (dbVar),RCVaccession,PhenotypeIDS,PhenotypeList,Origin,OriginSimple,Assembly,ChromosomeAccession,Chromosome,Start,Stop,ReferenceAllele,AlternateAllele,Cytogenetic,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF,label
0,15044,single nucleotide variant,NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter),55572,FOXRED1,HGNC:26927,Pathogenic,1,"Dec 30, 2019",267606829,-,RCV000000015|RCV000578659|RCV001194045,"MONDO:MONDO:0032624,MedGen:C4748791,OMIM:61824...","Mitochondrial complex 1 deficiency, nuclear ty...",germline,germline,GRCh38,NC_000011.10,11,126275389,126275389,na,na,11q24.2,"criteria provided, multiple submitters, no con...",3,-,N,"ClinGen:CA113792,OMIM:613622.0001",3,5,126275389,C,T,1
1,15074,single nucleotide variant,NM_001201543.2(FAM161A):c.685C>T (p.Arg229Ter),84140,FAM161A,HGNC:25808,Pathogenic,1,"Sep 15, 2021",267606794,-,RCV000000052|RCV001074032|RCV001090971|RCV0012...,"MONDO:MONDO:0011630,MedGen:C1419614,OMIM:60606...",Retinitis pigmentosa 28|Retinal dystrophy|not ...,germline,germline,GRCh38,NC_000002.12,2,61840319,61840319,na,na,2p15,"criteria provided, multiple submitters, no con...",6,-,N,"OMIM:613596.0001,ClinGen:CA251355",3,35,61840319,G,A,1
2,15075,single nucleotide variant,NM_001201543.2(FAM161A):c.1309A>T (p.Arg437Ter),84140,FAM161A,HGNC:25808,Pathogenic,1,"Sep 15, 2021",200691042,-,RCV000000053|RCV000153226|RCV000678572|RCV0007...,"MONDO:MONDO:0011630,MedGen:C1419614,OMIM:60606...",Retinitis pigmentosa 28|not provided|Cone-rod ...,germline;inherited;unknown,germline,GRCh38,NC_000002.12,2,61839695,61839695,na,na,2p15,"criteria provided, multiple submitters, no con...",19,-,N,"ClinGen:CA233978,OMIM:613596.0002",3,36,61839695,T,A,1
3,15077,single nucleotide variant,NM_001201543.2(FAM161A):c.1567C>T (p.Arg523Ter),84140,FAM161A,HGNC:25808,Pathogenic,1,"Apr 01, 2021",202193201,-,RCV000000055|RCV000787606|RCV000790648|RCV0010...,"MONDO:MONDO:0011630,MedGen:C1419614,OMIM:60606...",Retinitis pigmentosa 28|Retinal dystrophy|not ...,germline;inherited;unknown,germline,GRCh38,NC_000002.12,2,61839437,61839437,na,na,2p15,"criteria provided, multiple submitters, no con...",11,-,N,"ClinGen:CA233972,OMIM:613596.0004",3,38,61839437,G,A,1
4,15099,single nucleotide variant,NM_006642.5(SDCCAG8):c.740+356C>T,10806,SDCCAG8,HGNC:10671,Pathogenic,1,"Jun 01, 2018",397515337,-,RCV000000077|RCV000760978,"MONDO:MONDO:0014444,MedGen:C3889474,OMIM:61599...",Bardet-Biedl syndrome 16|Senior-Loken syndrome 7,germline,germline,GRCh38,NC_000001.11,1,243305133,243305133,na,na,1q43,"criteria provided, multiple submitters, no con...",3,-,N,"ClinGen:CA251367,OMIM:613524.0004",3,60,243305133,C,T,1


In [5]:
variant_df['ClinicalSignificance'].value_counts()

Benign        29927
Pathogenic    10298
Name: ClinicalSignificance, dtype: int64

In [8]:
extra_len = 100
res_file_path = "../data/seq/{}/seq.csv".format(2 * extra_len + 1)
if not os.path.exists(os.path.dirname(res_file_path)):
    os.makedirs(os.path.dirname(res_file_path))

In [ ]:
variant_df[['seq_ref', 'seq_alt']] = variant_df.parallel_apply(get_seq, result_type='expand', axis=1, seq_len=extra_len)

In [ ]:
variant_df[['ReferenceAlleleVCF', 'AlternateAlleleVCF', 'seq_ref', 'seq_alt', 'label']].to_csv(res_file_path, index=False)

In [15]:
variant_df.head(5)

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),nsv/esv (dbVar),RCVaccession,PhenotypeIDS,PhenotypeList,Origin,OriginSimple,Assembly,ChromosomeAccession,Chromosome,Start,Stop,ReferenceAllele,AlternateAllele,Cytogenetic,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF,label,seq_ref,sequence
0,15044,single nucleotide variant,NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter),55572,FOXRED1,HGNC:26927,Pathogenic,1,"Dec 30, 2019",267606829,-,RCV000000015|RCV000578659|RCV001194045,"MONDO:MONDO:0032624,MedGen:C4748791,OMIM:61824...","Mitochondrial complex 1 deficiency, nuclear ty...",germline,germline,GRCh38,NC_000011.10,11,126275389,126275389,na,na,11q24.2,"criteria provided, multiple submitters, no con...",3,-,N,"ClinGen:CA113792,OMIM:613622.0001",3,5,126275389,C,T,1,CTCATCCCTCTTTGTGAGTTCTCTTTTTCTTATCACAGGGATGGAG...,CTCATCCCTCTTTGTGAGTTCTCTTTTTCTTATCACAGGGATGGAG...
1,15074,single nucleotide variant,NM_001201543.2(FAM161A):c.685C>T (p.Arg229Ter),84140,FAM161A,HGNC:25808,Pathogenic,1,"Sep 15, 2021",267606794,-,RCV000000052|RCV001074032|RCV001090971|RCV0012...,"MONDO:MONDO:0011630,MedGen:C1419614,OMIM:60606...",Retinitis pigmentosa 28|Retinal dystrophy|not ...,germline,germline,GRCh38,NC_000002.12,2,61840319,61840319,na,na,2p15,"criteria provided, multiple submitters, no con...",6,-,N,"OMIM:613596.0001,ClinGen:CA251355",3,35,61840319,G,A,1,ATATCTGATTTAGATTTCATGGACTCTTCTTTTTTCTTCTGTTCTC...,ATATCTGATTTAGATTTCATGGACTCTTCTTTTTTCTTCTGTTCTC...
2,15075,single nucleotide variant,NM_001201543.2(FAM161A):c.1309A>T (p.Arg437Ter),84140,FAM161A,HGNC:25808,Pathogenic,1,"Sep 15, 2021",200691042,-,RCV000000053|RCV000153226|RCV000678572|RCV0007...,"MONDO:MONDO:0011630,MedGen:C1419614,OMIM:60606...",Retinitis pigmentosa 28|not provided|Cone-rod ...,germline;inherited;unknown,germline,GRCh38,NC_000002.12,2,61839695,61839695,na,na,2p15,"criteria provided, multiple submitters, no con...",19,-,N,"ClinGen:CA233978,OMIM:613596.0002",3,36,61839695,T,A,1,TCTCTTTTAATAGATGCATGTGGAGATGCATGAAGATCAAATGGTT...,TCTCTTTTAATAGATGCATGTGGAGATGCATGAAGATCAAATGGTT...
3,15077,single nucleotide variant,NM_001201543.2(FAM161A):c.1567C>T (p.Arg523Ter),84140,FAM161A,HGNC:25808,Pathogenic,1,"Apr 01, 2021",202193201,-,RCV000000055|RCV000787606|RCV000790648|RCV0010...,"MONDO:MONDO:0011630,MedGen:C1419614,OMIM:60606...",Retinitis pigmentosa 28|Retinal dystrophy|not ...,germline;inherited;unknown,germline,GRCh38,NC_000002.12,2,61839437,61839437,na,na,2p15,"criteria provided, multiple submitters, no con...",11,-,N,"ClinGen:CA233972,OMIM:613596.0004",3,38,61839437,G,A,1,AAATTTACCAACATAAACTCCACAGAGCTGCATACACTCATCTGGC...,AAATTTACCAACATAAACTCCACAGAGCTGCATACACTCATCTGGC...
4,15099,single nucleotide variant,NM_006642.5(SDCCAG8):c.740+356C>T,10806,SDCCAG8,HGNC:10671,Pathogenic,1,"Jun 01, 2018",397515337,-,RCV000000077|RCV000760978,"MONDO:MONDO:0014444,MedGen:C3889474,OMIM:61599...",Bardet-Biedl syndrome 16|Senior-Loken syndrome 7,germline,germline,GRCh38,NC_000001.11,1,243305133,243305133,na,na,1q43,"criteria provided, multiple submitters, no con...",3,-,N,"ClinGen:CA251367,OMIM:613524.0004",3,60,243305133,C,T,1,gcggtggctcacgcctgtaatcccagcactttgggaggctgaagcg...,GCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCTGAAGCG...


In [14]:
"""
突变后的序列
"""
variant_df.rename(columns={'seq_alt' : 'sequence'}, inplace=True)
variant_df['sequence'] = variant_df['sequence'].apply(lambda x: x.upper())
variant_df[['sequence', 'label']].to_csv(res_file_path, index=False)

In [40]:
"""
split train test dataset
"""

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os

data_dir = '../data/seq/201/5/'
data_file_path = '../data/seq/201/5/kmers5.csv'

print("processing {}".format(data_file_path))
data = pd.read_csv(data_file_path, delimiter='\t')
data = data.values
seqs, label = data[:, 0], data[:, 1]

train_data, val_test_data, train_label, val_test_label = train_test_split(seqs, label, train_size=0.7, random_state=0, stratify=label) 
print(train_data.shape, val_test_data.shape, train_label.shape, val_test_label.shape)

val_data, test_data, val_label, test_label = train_test_split(val_test_data, val_test_label, test_size=0.5, random_state=0, stratify=val_test_label)
print(val_data.shape, val_label.shape, test_data.shape, test_label.shape)

train_dataset = np.column_stack((train_data, train_label))
val_dataset = np.column_stack((val_data, val_label))
test_dataset = np.column_stack((test_data, test_label))

train_dataset = np.insert(train_dataset, 0, ['sequence', 'label'], axis=0)
val_dataset = np.insert(val_dataset, 0, ['sequence', 'label'], axis=0)
test_dataset = np.insert(test_dataset, 0, ['sequence', 'label'], axis=0)


train_dir = os.path.join(data_dir, 'train')
if not os.path.exists(train_dir):
    os.makedirs(train_dir)


np.savetxt(os.path.join(train_dir, 'train.tsv'), train_dataset, delimiter='\t', fmt='%s')
np.savetxt(os.path.join(train_dir, 'dev.tsv'), val_dataset, delimiter='\t', fmt='%s')

test_dir = os.path.join(data_dir, 'test')
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

np.savetxt(os.path.join(test_dir, 'dev.tsv'), test_dataset, delimiter='\t', fmt='%s')

In [37]:
from sklearn.model_selection import StratifiedShuffleSplit

In [14]:
import numpy as np

result  = np.load('../result/mutation/pred_results.npy')


In [15]:
result.shape

(6034,)

In [18]:
import pandas as pd


test_data = pd.read_csv('../data/seq/201/6/test/dev.tsv', sep='\t')

In [25]:
truths = test_data['label'].to_numpy()

In [29]:
from collections import Counter

counter = Counter(truths)
counter

Counter({0: 4489, 1: 1545})

In [33]:
from sklearn.metrics import classification_report,confusion_matrix

preds = np.where(result<=0.5, 0, 1)
print(classification_report(truths, preds))

              precision    recall  f1-score   support

           0       0.83      0.90      0.87      4489
           1       0.62      0.48      0.54      1545

    accuracy                           0.79      6034
   macro avg       0.73      0.69      0.70      6034
weighted avg       0.78      0.79      0.78      6034



In [34]:
confusion_matrix(truths, preds)

array([[4045,  444],
       [ 806,  739]])